## 예술의전당 콘서트홀 가격 모델

### 1. 라이브러리

In [4]:
# base
import pandas as pd
import numpy as np
import re

# files
from glob import glob

# visualization
import matplotlib.pyplot as plt
import koreanize_matplotlib
import plotly.express as px
import seaborn as sns

In [5]:
# settings
%matplotlib inline
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_rows', 100)


### 2. 데이터셋 로드

In [2]:
def load_file (file_name, file_num=0):
    files = glob(file_name)
    file = files[file_num]   
    df = pd.read_csv(files[file_num])

    print(f"Found.. {len(files)} file(s) : {files}")    
    print(f"Reading.. {files[file_num]}")
    print(f"dataframe.shape : {df.shape}")
    
    return df

In [50]:
# df : 예매데이터 + 좌석데이터 merge된 데이터셋
df = load_file('클래식_빈좌석포함데이터.csv')
df = df[(df['place']=='콘서트홀') & (df['genre']=='클래식')]
print(f"df_book.shape : {df.shape}")

/var/folders/48/rp3jrl0n4d114g_6b09p39700000gn/T/ipykernel_7821/2598521325.py:4: DtypeWarning: Columns (19,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(files[file_num])


Found.. 1 file(s) : ['클래식_빈좌석포함데이터.csv']
Reading.. 클래식_빈좌석포함데이터.csv
dataframe.shape : (405810, 42)
df_book.shape : (405810, 42)


### 3. 원가격추정-1 : 판매된 좌석의 할인 전 금액 추정하기

discount_type 에서 '할인율' 혹은 '할인액'을 구해서 'price'에서 역산한다.

In [51]:
# discount_type에 적용된 할인의 종류가 다양함.
df['discount_type'].value_counts()

초대권            93749
일반             22264
기획사판매          21029
골드회원 할인10%      6634
그린회원 할인5%       3307
               ...  
그린회원 할인30%         1
노블회원 할인50%         1
특판B 30%            1
기획사할인50%           1
골드회원 할인10%_        1
Name: discount_type, Length: 213, dtype: int64

In [52]:
# 할인율 계산 > % 앞의 숫자만 추출
df['할인율'] = df['discount_type'].str.extract('(\d+)%')
df['할인율'] = df['할인율'].fillna(0).astype(int)
df['할인율'] = df['할인율'] / 100
print(df['할인율'].value_counts().sort_index())

# NaN인 경우 (티켓이 판매되지 않은 경우) 채워넣기
df[['할인율', 'price']] = df[['할인율', 'price']].fillna(0)

0.00    368076
0.05      9230
0.10     10125
0.12       640
0.15       974
0.20      4371
0.25       372
0.30      5351
0.40      3597
0.50      3074
Name: 할인율, dtype: int64


In [53]:
# discount_type에 할인율이 명시되어 있지 않은 경우 > 대부분 초대권, 기획사판매, 예매권
df.loc[df['할인율']==0, 'discount_type'].value_counts()

초대권                  93749
일반                   22264
기획사판매                21029
공연진행석                 2406
홍보진행                  1131
당일할인티켓                 981
기획사할인                  695
차액                     339
중앙일보 JTBC              200
당일할인티켓_                116
중앙일보 JTBC 초대권          100
공연예매권                   88
기획사                     18
하비에르 국제학교 학부모, 직원       13
수험생 할인(동반1인)            10
하비에르 국제학교 재학생            9
싹틔우미 할인                  3
Name: discount_type, dtype: int64

In [54]:
# '할인전금액' 구하기 : 'price(판매금액)'에 '할인율'을 역산
df['할인전금액'] = df['price'] / (1 - df['할인율'])
df['할인전금액'] = df['할인전금액'].astype(int)

df[['discount_type', 'price', '할인율', '할인전금액']]

,discount_type,price,할인율,할인전금액
0,기획사판매,0.0,0.00,0
1,기획사판매,0.0,0.00,0
2,기획사판매,0.0,0.00,0
3,기획사판매,0.0,0.00,0
4,기획사판매,0.0,0.00,0
...,...,...,...,...
405805,장애인/국가유공자 할인50%,15000.0,0.50,30000
405806,장애인/국가유공자 할인50%,15000.0,0.50,30000
405807,그린회원 할인5%,28000.0,0.05,29473
405808,NaN,0.0,0.00,0


In [158]:
# 입장권이 5천원도 안되는 금액일 때 => 이상치 같은데 0원으로 처리?
df.loc[(df['할인전금액'] < 5000) & (df['할인전금액'] != 0), '할인전금액'] = 0

### 4. 원가격추정-2 : 판매되지 않은 좌석의 티켓값 구하기

In [57]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [159]:
booked_rate_dict = {}  # 정답을 아는 데이터의 비율
# n_neighbors_dict = {}  # 
best_n_neighbors_dict = {}  # 
best_mse_dict = {}
공연별_df_dict = {}

for time in df['전체공연시간'].unique():
    공연별_df = df[df['전체공연시간'] == time]

    # 예매된 경우, 예매되지 않은 경우 구분
    booked_seat = 공연별_df[공연별_df['할인전금액'] > 0]
    emppty_seat = 공연별_df[공연별_df['할인전금액'] == 0].drop('할인전금액', axis=1)
    booked_rate = round(booked_seat.shape[0] / 공연별_df.shape[0], 3)
    booked_rate_dict[time] = booked_rate

    if booked_rate ==0:
        continue

    # == booked_seat을 이용해서 적절한 n-neighbors값 찾기 ==
    # X, y 
    X = booked_seat[['X', 'Y', 'Z']]
    y = booked_seat['할인전금액']

    # # train-valid
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

    # n-neighbors값 찾기
    best_n_neighbors = None
    best_mse = float('inf')
    for n in range(1, min(50, X_train.shape[0]) + 1):
        knn = KNeighborsRegressor(n_neighbors=n, weights='distance', p=2)
        knn.fit(X_train, y_train)
        y_valid_pred = knn.predict(X_valid)
        mse = round(mean_squared_error(y_valid, y_valid_pred))
        if mse < best_mse:
            best_mse = mse
            best_n_neighbors = n
    
    # 공연별로 best n-neighbors값, mse값 저장하기
    best_n_neighbors_dict[time] = best_n_neighbors
    best_mse_dict[time] = best_mse
    print(f"best_n_neighbors : {best_n_neighbors}, best_mse : {best_mse}")

    # n-neighbors값 적용해서 원가격 추정하기
    knn = KNeighborsRegressor(n_neighbors=best_n_neighbors, weights='distance', p=2)
    knn.fit(X, y)
    emppty_seat['할인전금액'] = np.round(knn.predict(emppty_seat[['X', 'Y', 'Z']]), -2).astype(int)

    공연별_df = pd.concat([booked_seat, emppty_seat], axis=0).sort_values(by='seat')
    공연별_df = 공연별_df.reset_index().drop('index', axis=1)
    공연별_df_dict[time] = 공연별_df


best_n_neighbors : 3, best_mse : 27940690
best_n_neighbors : 1, best_mse : 0
best_n_neighbors : 1, best_mse : 1000000
best_n_neighbors : 1, best_mse : 0
best_n_neighbors : 3, best_mse : 26107387
best_n_neighbors : 8, best_mse : 6078335
best_n_neighbors : 2, best_mse : 39005478
best_n_neighbors : 2, best_mse : 72045451
best_n_neighbors : 1, best_mse : 38455
best_n_neighbors : 5, best_mse : 9746948
best_n_neighbors : 4, best_mse : 51983389
best_n_neighbors : 1, best_mse : 54191
best_n_neighbors : 1, best_mse : 0
best_n_neighbors : 1, best_mse : 0
best_n_neighbors : 1, best_mse : 15311061
best_n_neighbors : 3, best_mse : 25655831
best_n_neighbors : 3, best_mse : 18894622
best_n_neighbors : 2, best_mse : 46115454
best_n_neighbors : 6, best_mse : 58134882
best_n_neighbors : 2, best_mse : 314982905
best_n_neighbors : 1, best_mse : 40000
best_n_neighbors : 9, best_mse : 146764
best_n_neighbors : 1, best_mse : 6451613
best_n_neighbors : 1, best_mse : 28407703
best_n_neighbors : 2, best_mse : 8

In [160]:
# 총 162개 공연 중
print(f"총 {df['전체공연시간'].nunique()} 공연 중..")
print(f"가격을 알 수 없는 공연 : {sum([1 for i, j in booked_rate_dict.items() if j==0])}개")
print(f"가격을 알 수 있는 공연 : {sum([1 for i, j in booked_rate_dict.items() if j!=0])}개")

new_df = pd.concat(list(공연별_df_dict.values()), axis=0, ignore_index=True)
new_df.shape

총 162 공연 중..
가격을 알 수 없는 공연 : 11개
가격을 알 수 있는 공연 : 151개


(378255, 44)

In [161]:
new_df['할인전금액'].describe()

count    378255.000000
mean      76715.542875
std       46454.892610
min        5000.000000
25%       45000.000000
50%       70000.000000
75%      100000.000000
max      320000.000000
Name: 할인전금액, dtype: float64

In [171]:
temp = list(공연별_df_dict.values())[17]

fig = px.scatter_3d(temp, x='X', y='Y', z='Z', color='할인전금액', hover_name='seat')
fig.update_layout(width=800, height=600)
fig.update_traces(marker={'size': 1})
fig.show()